# Parameters

In [1]:
GROUP_NAME = "RTDs" # just a title for use in charts
BRAND_NAME = "Shine+"
CSV_EXPORT_NAME = "Shine+"

In [2]:
brand_name_di = {}

brand_name_di['https://www.shinedrink.com/'] = 'Shine+'
brand_name_di['http://rowdyenergy.com/'] = 'Rowdy Energy'
brand_name_di['http://drinkolipop.com/'] = 'Olipop'
brand_name_di['https://drinkpoppi.com/'] = 'Poppi'
brand_name_di['https://www.drinkaha.com/'] = 'Aha'

brand_name_di

{'https://www.shinedrink.com/': 'Shine+',
 'http://rowdyenergy.com/': 'Rowdy Energy',
 'http://drinkolipop.com/': 'Olipop',
 'https://drinkpoppi.com/': 'Poppi',
 'https://www.drinkaha.com/': 'Aha'}

In [3]:
import re

def stand_url(url):
    # this function searches for content after 'www.' and before next '/'
    # If 'www.' doesn't exist, then it will search for '//' and return the content after '//' and before next '/'
    # if both 'www.' and '//' don't exist,  then it will return original url before next '/'
    if 'www.' in url:
        pattern = re.compile(r'(?<=www.)(?:(?!/).)*')
        result = pattern.search(url)
        return result.group()
    elif '//' in url:
        pattern = re.compile(r'(?<=//)(?:(?!/).)*')
        result = pattern.search(url)
        return result.group()
    else:
        pattern2 = re.compile(r'(?:(?!/).)*')
        result2 = pattern2.search(url) 
        return result2.group()


In [4]:
# Normalize URLs
brand_name_di = {stand_url(key): value for key, value in brand_name_di.items()}
brand_name_di

{'shinedrink.com': 'Shine+',
 'rowdyenergy.com': 'Rowdy Energy',
 'drinkolipop.com': 'Olipop',
 'drinkpoppi.com': 'Poppi',
 'drinkaha.com': 'Aha'}

In [5]:
# Do you want to print all dataframes for QA?
# WARNING: will make your script run for a longer time

# 'Y' for Yes
# 'N' or empty for No

print_all_dataframes = 'Y'

In [6]:
# typically 100 - judgement call to increase/decrease
# any lower than 80 might avoid 
MIN_REV_COUNT = 80 # Unique reviewers required for analysis (closedly related to but not quite reviews)

add/subtract kpcs from this list https://docs.google.com/spreadsheets/d/1jjexRCEuARAdghPv3FCoyZtJ6CuWBUG3vTofyGxQf3A/edit#gid=0 

This list was generated based on topic modeling in early 2020

In [7]:
# Note: review above google sheets KPC list (topics per category) to determine KPC words to add/exclude
# KPC google sheet created Feb 2021 

# Add if relevant for brand
ADD_KPC_LI = [
    'Flavor',
    'Value for Money',
    'Efficacy',
    'Natural',
    'Nutritionals'
] # 

# Exclude if not relevant for brand
EXCLUDE_KPC_LI = [
#                 'Easy to use',
#                  'Packaging'
]

# Determine closest category in Google Sheet to run for brand
# Can also pull from diff categories, if the seedwords make sense for brand
PRIMARY_CATEGORY_LI = [] # could be custom

In [8]:
GROUP_NAME = "RTDs" # just a title for use in charts

# additional words to filter before combining into ngrams
CUSTOM_STOPWORDS = []

In [9]:
# SENT - sentences
# regex to break out sentences
CUSTOM_SENT_SPLIT = "[..|...|....|;|-]"

# min length - reviews with substance
MIN_SENT_LENGTH = 3

In [10]:
#csv_input =  input("Enter all CSV locations EXACTLY as displayed in your repo, separated by a comma and space. \nIf not adding CSVs, leave blank \n \n").strip()

In [11]:
# if a brand sells more than just one type of product, should focus on the main product
# format: dict of string (comma separated lower case words) in order matching that in NORMALIZED_URL_LI
# if there is a string for one brand, must enter empty strings for all others to keep matching order.
# in the future this can be replaced potentially by product ER (category or attributes)

# multiple words for one brand 'farm to flock, layena'
# make sure to pay attention to ordering (match brand_name_di order)

PRODUCT_NAME_CONTAINS_LI = ['',
                            '',
                            '',
                            '',
                            ''
] # the csv list is an OR operation, it includes product with ANY of the listed words

PRODUCT_NAME_NOT_CONTAINS_LI = [
    '',
    '',
    '',
    '',
    ''
] # the csv list is an AND operation, it excludes product with ANY of the listed words

In [12]:
TOP_P = 20 # top words per aspect to potentially add per iteration

In [13]:
# cols to query in main reviews dataset
REV_COLS = [
    'source_name',
    'normalized_url',
    'review_source_id',
    'reviewer_source_id',
    'brand_name',
    'product_name',
    'product_source_id',
    'review_date',
    'review_rating',
    'review_content'
]

# cols to query in later cell
GROUP_COLS = [
    'normalized_url',
    'review_source_id',
    'review_content',
#   'source_name', # actually some of them are duplicates across sources
    'top_asp',
    ]

In [14]:
NORMALIZED_URL_LI = list(brand_name_di.keys())
BRAND_NAME_LI = list(brand_name_di.values())

In [15]:
NORMALIZED_URL_LI

['shinedrink.com',
 'rowdyenergy.com',
 'drinkolipop.com',
 'drinkpoppi.com',
 'drinkaha.com']

In [16]:
BRAND_NAME_LI

['Shine+', 'Rowdy Energy', 'Olipop', 'Poppi', 'Aha']

In [17]:
product_name_include_di = dict(zip(NORMALIZED_URL_LI, [w.split(", ") if w else [] for w in PRODUCT_NAME_CONTAINS_LI ]))
product_name_exclude_di = dict(zip(NORMALIZED_URL_LI, [w.split(", ") if w else [] for w in PRODUCT_NAME_NOT_CONTAINS_LI]))

In [18]:
# Check outputs below to make sure product inclusion/exclusions is assigned as intended
product_name_include_di

{'shinedrink.com': [],
 'rowdyenergy.com': [],
 'drinkolipop.com': [],
 'drinkpoppi.com': [],
 'drinkaha.com': []}

In [19]:
# Check outputs below to make sure product inclusion/exclusions is assigned as intended
product_name_exclude_di 

{'shinedrink.com': [],
 'rowdyenergy.com': [],
 'drinkolipop.com': [],
 'drinkpoppi.com': [],
 'drinkaha.com': []}

# Assumes Spark 2.4

Some array function only in spark >2.3. I use docker image with statsmodels and plotly and vaderSentiment based on spark 2.4.5
Please deloy cluster with `--docker-tag cici-nlp-sentiment` or other spark 2.4 docker image.

In [21]:
sc

<SparkContext master=spark://terry.cu:7077 appName=PySparkShell>

# Imports

In [27]:
%%capture
!pip install watermark
!pip3 install --upgrade --no-cache-dir --extra-index-url http://pypi.cu/root/circleup/+simple/ --trusted-host pypi.cu cu-helio-insights==0.0.12
!pip install -Iv numpy==1.19.2
!pip install ipynb
!pip install

In [28]:
%load_ext watermark
%watermark -v -m --iversions -g

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
py4j     0.10.7
re       2.2.1
platform 1.0.8
CPython 3.6.8
IPython 7.16.1

compiler   : GCC 6.3.0 20170516
system     : Linux
release    : 4.9.43-17.39.amzn1.x86_64
machine    : x86_64
processor  : 
CPU cores  : 4
interpreter: 64bit
Git hash   :


In [33]:
import numpy as np
import pandas as pd
from pyspark.sql.window import Window
#from pyspark.sql.functions import arrays_zip
from pyspark.ml.feature import CountVectorizer, IDF, HashingTF
from pyspark.ml.pipeline import Pipeline
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.util import bigrams
from nltk.stem import WordNetLemmatizer
from nltk import sent_tokenize
import string
import plotly
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.express as px
from plotly.subplots import make_subplots
from scipy import stats
from sklearn import linear_model

import statsmodels.api as sm
from itertools import chain
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer as SIA

import insights
from insights.investor_tools.widgets.style import (
    Font,
    CU_PLOTLY_COLOR_SEQUENCE,
    CU_PLOTLY_COLORSCALE,
    Color
)
from spark_tools import T, F, c, read_google_sheet
import apollo
from apollo import OverrideConfiguration
from apollo import dataset
import apollo_artifacts
from apollo_artifacts import datasets
from transform.attributes.utils import tf_idf_attributes

import builtins

ImportError: Something is wrong with the numpy installation. While importing we detected an older version of numpy in ['/usr/local/lib/python3.6/site-packages/numpy']. One method of fixing this is to repeatedly uninstall numpy until none is found, then reinstall this version.

In [ ]:
## SUMMARY
# Importing aspects of interest from KPC google sheet

bootstrap_dict = {}

from spark_tools import T, F, c, read_google_sheet
GSHEET_LOC = "https://docs.google.com/spreadsheets/d/1jjexRCEuARAdghPv3FCoyZtJ6CuWBUG3vTofyGxQf3A/"
GSHEET_NAME = 'top_per_category'

# primary category
top_seedwords_sdf = read_google_sheet(GSHEET_LOC, sheet=GSHEET_NAME)

# place for the analyst to manually add?
ADD_GSHEET_NAME = 'kpc_seedword_map'
# seedwords
add_seedword_sdf = read_google_sheet(GSHEET_LOC, sheet=ADD_GSHEET_NAME)

# finding the primary category and terms here
aspect_sdf = top_seedwords_sdf.filter(F.col('primary_category').isin(PRIMARY_CATEGORY_LI)).select('kpc_name', 'seed_words')

# add any of the manually specified criteria here
add_aspect_sdf =  add_seedword_sdf.filter(F.col('kpc_name').isin(ADD_KPC_LI)).select('kpc_name', 'seed_words')

base_aspect_dict = aspect_sdf.union(add_aspect_sdf).rdd.map(lambda x : (x[0], x[1].split(', '))).collectAsMap()

# remove any we specified as not wanting to keep
for asp in EXCLUDE_KPC_LI:
    del base_aspect_dict[asp]

# Import stopwords and functions .ipynb from HAI General S3

Much of the logic for pulling and cleaning reviews is shared between TF-IDF and KPC scripts

In [ ]:
os.getcwd()

In [ ]:
# cd to where the helpers and config files are located
%cd /circleup-notebooks/s3-circleup-notebooks/HAI/General/Code/TF_IDF

from ipynb.fs.full.functions_tfidf import *

# cd - command changes back to previously defined directory (should be same as og_filepath)
%cd /circleup-notebooks/s3-circleup-notebooks/SPan/KPC
%pwd

In [ ]:
# cd to where the helpers and config files are located
%cd /circleup-notebooks/s3-circleup-notebooks/HAI/General/helpers

from ipynb.fs.full.general_helpers import *

# cd - command changes back to previously defined directory (should be same as og_filepath)
%cd /circleup-notebooks/s3-circleup-notebooks/SPan/KPC
%pwd

# Settings

In [ ]:
np.random.seed(42) # set seed for models for reproducibility

stopwords_li = stopwords.words('english')
punkts_li = list(string.punctuation)

pd.options.display.max_rows = 300
OverrideConfiguration(default_to_production=True).apply()

spark.conf.set('spark.sql.execution.arrow.enabled', 'false')

In [ ]:
# Issues adding to external helpers file
# Why? - when you use F.udf, this is a "whole new instance" so you wont be able to use any of the stuff you just imported
# Why do we need @F.udf? Unsure
# NLTK package has sent_tokenize_udf, why don't we use this?
     # Specific use case for Spark data? vs. Python data
     # Using python function on spark data
@F.udf(T.ArrayType(T.StringType()))
def sent_tokenize_udf(s):
    return sent_tokenize(s)

# Load Data

In [ ]:
from datetime import *
from datetime import datetime, timedelta


test_timeline_data = (
apollo
.dataset('online_reviews__deduped_review__1_0')
.read()
.select('normalized_url', 'execution_date')
.where(c.execution_date > date.today() - timedelta(days=7))
)

max_date = test_timeline_data.where(c.normalized_url.isNotNull()).agg({'execution_date':'max'}).collect()[0][0]

reviews = (
dataset('online_reviews__deduped_review__1_0')
.read_segment_df({'execution_date': max_date})
.select(REV_COLS)
.persist()
)

In [ ]:
panda(reviews)

## Check to make sure 'normalized_url' is coming through properly

In [ ]:
panda(reviews)['normalized_url'].head(5)

# Manual Tweaking for Potential ER Issues
- If we have reviews for a brand, but the URL is incorrect, we can fix it below
- Manually fix brand id --> normalized url mapping issues

In [ ]:
ER_BRAND_URL_MAP = {'thepoppy.store': 'drinkpoppi.com'}
# ER_BRAND_URL_MAP = {'dae':'daehair.com'} # manually add {brand_id : normalized_url}

URL_BLACKLIST = []
# Some urls sneak into the list somehow. Add them here to remove them from the charts

for bad_url, url in ER_BRAND_URL_MAP.items():
    reviews = reviews.withColumn(
        "normalized_url",
        F.when(
            F.col("normalized_url") == bad_url,
            url
        ).otherwise(F.col("normalized_url"))
    )

# Filter for Minimal Rev Count

In [ ]:
%%time
reviews.filter(F.col('normalized_url').isin(NORMALIZED_URL_LI)).groupby('normalized_url').count().limit(20).toPandas().sort_values('count', ascending = False)

In [ ]:
%%time
reviews.filter(F.col('normalized_url').isin(NORMALIZED_URL_LI)).groupby('normalized_url','source_name').count().limit(20).toPandas().sort_values('count', ascending = False)

In [ ]:
# Make sure URL meets min rev count

count_revs = (
    reviews
    .dropna(subset=['normalized_url'])
    .groupby('normalized_url')
    .agg(
        F.countDistinct(F.concat(F.col('source_name'), F.col('reviewer_source_id'))).alias('reviewer_count')
    )
)
    
count_revs_filtered = (
    count_revs
    .filter(F.col('reviewer_count') >= MIN_REV_COUNT)
    .select('normalized_url')
    .distinct()
)

filtered_revs = (
    reviews
    .join(count_revs_filtered, on='normalized_url')
)

## Check in Reviews with Enough Reviews of Reviews at All
Check for ER misses here which occur frequently

In [ ]:
brand_revs = filtered_revs.filter(F.col('normalized_url').isin(NORMALIZED_URL_LI))

In [ ]:
%%time
(reviews
 .filter(F.col('normalized_url').isin(NORMALIZED_URL_LI))
 .cache()
 .groupby('normalized_url','product_name')
 .count()
 .sort('count', ascending=False)
 .limit(10).toPandas())
 
#  .show(5, truncate=False))

## Optional Filter for Product
- include/exclude products

In [ ]:
if PRODUCT_NAME_CONTAINS_LI:
    brand_revs = filter_included_products(brand_revs, product_name_include_di, "product_name", "normalized_url")

if PRODUCT_NAME_NOT_CONTAINS_LI:
    brand_revs = filter_excluded_products(brand_revs, product_name_exclude_di, "product_name", "normalized_url")

## Optional Filter for Review Content

Use Cases: if there is a mapping issue or need to add a word filter on review content, use this function to exclude reviews. 

In [ ]:
exclude_word_li = [
    '',
    '',
    '',
    ''
]

In [ ]:
brand_revs = brand_revs.where(
    ~brand_revs['review_content'].rlike("|".join(["(" + pat + ")" for pat in exclude_word_li]))
)

## check most common products to see if anything else to exclude; if yes go back up to product exclusion


In [ ]:
%%time
(brand_revs
.filter(F.col('normalized_url').isin(NORMALIZED_URL_LI))
.cache()
.groupby('normalized_url','product_name')
.count()
.sort('count', ascending=False)
.limit(30).toPandas())

 
#  .show(50, truncate=False))

# Upload CSV
- DTC CSV input
- Cases where we manually scrape DTC reviews, then input the CSV here

In [ ]:
external_revs_list = []

for i in range(len(csv_input.split(', '))):
    if csv_input != '':
        external_revs_list.append(pd.read_csv((csv_input.split(', ')[i])))

external_revs_list[0]['review_rating'] = external_revs_list[0]['review_rating'].str.replace('\n','')
external_revs_list[0]['review_rating'] = external_revs_list[0]['review_rating'].str.slice(6,7)
external_revs_list[0]

In [ ]:
# combining each of the external review lists
from pyspark.sql.types import StructType
mySchema = StructType(brand_revs.schema)
if external_revs_list != []:
    cleaned_external_review_list = []
    for external_revs_csv in external_revs_list:  
        external_revs_csv.rename(columns={'brand':'brand_name'},inplace=True)
        external_revs_csv['review_rating']=pd.to_numeric(external_revs_csv['review_rating'], downcast='float')

        external_revs_csv['source_name'] = 'DTC'
        # Checking for static columns, to add blanksto data frame
        # Then checking for incrementing columns, which we'll set as the index, assuming they are unique reviewers and reviews
        static_columns = ['product_name']
        # Add or remove columns that need to be populated, usually will be review_source_id, reviewer_source_id, or product_source_id if needed
        incrementing_columns = ['review_source_id','reviewer_source_id', 'product_source_id']

        for column_name in static_columns:
            if column_name not in external_revs_csv:
                external_revs_csv[column_name] = ''
            else:
                pass
        for column_name in incrementing_columns:
            external_revs_csv[column_name]=external_revs_csv.index

        # selecting the same columns as internal reviews CSV
        external_revs_csv = external_revs_csv[reviews.columns]
        cleaned_external_review_list.append(external_revs_csv)

    external_revs_csv = pd.concat(cleaned_external_review_list)
    external_revs_csv = external_revs_csv[['normalized_url','source_name','review_source_id','reviewer_source_id','brand_name','product_name','product_source_id','review_date','review_rating','review_content']]
    external_revs_sdf = spark.createDataFrame(external_revs_csv, schema = mySchema)

In [ ]:
# combine the and clean review_content of dataframes, which should at this point have matching columns
if external_revs_list != []:
    brand_revs = brand_revs.union(external_revs_sdf)

brand_revs = brand_revs.withColumn(
    'review_content', remove_special_characters("review_content")).alias('review_content')

# removing hypertext from reviews
brand_revs = brand_revs.withColumn(
    'review_content', F.regexp_replace(
        F.regexp_replace("review_content", r"(?i)<a\s*[^>]*>", ""), r"(?i)<\s*/\s*a\s*>", ""
    ).alias('review_content'))

# removing special characters from reviews
brand_revs = brand_revs.withColumn(
    'review_content', remove_special_characters("review_content")).alias('review_content')


In [ ]:
!pip install contractions
import contractions
# limiting because analysis can slow way down with too many reviews
REVIEWS_PER_BRAND_TO_ANALYZE = 10_000

#get most recent n reviews 
# strip contractions
brand_revs_pdf = brand_revs.toPandas()
brand_revs_pdf = (brand_revs_pdf
                  .sort_values('review_date',ascending = False)
                  .groupby('normalized_url')
                  .head(REVIEWS_PER_BRAND_TO_ANALYZE))
brand_revs_pdf.to_csv(BRAND_NAME+'_&comps_reviews.csv', index=False)
#brand_revs_pdf['review_content'] = brand_revs_pdf['review_content'].apply(lambda x: [contractions.fix(word) for word in x.split()])

brand_revs = spark.createDataFrame(brand_revs_pdf)

In [ ]:
brand_revs.groupby('normalized_url').count().show()

# Start Iterative Bootstrapping
This is the start of iterative process. If seedwords are sufficient, no need to run the below more than once. If adding to seedwords, once enough words are added, run one last time from here.

In [ ]:
aspect_dict = {key: value + bootstrap_dict.get(key, []) for key, value in base_aspect_dict.items()}

# Tokenize to Sentences

In [ ]:
brand_rev_sents = tokenize_to_sentences(brand_revs, 'tokenized_sents', 'review_content', CUSTOM_SENT_SPLIT, MIN_SENT_LENGTH)

# Tokenize to Words

In [ ]:
token_brand_revs = (
    brand_rev_sents
    .withColumn('tokenized_text', lemmatize(tokenize(F.col('tokenized_sents'), F.lit(' '.join(CUSTOM_STOPWORDS)))))
    .dropna(subset=['tokenized_text'])
)

unigram_sdf = tokenize_to_ngrams(token_brand_revs, 'uni', 'tokenized_word', 'tokenized_text')
bigram_sdf = tokenize_to_ngrams(token_brand_revs, 'bi', 'bigram_word', 'tokenized_text')
unibi_sdf = tokenize_to_ngrams(token_brand_revs, 'unibi', 'unibi_word', 'tokenized_text')

# Aspect Segmentation

## Map

This maps tokenized words in reviews to the "aspects" or purchase criteria

In [ ]:
ASPECT_DICT = aspect_dict

aspect_li = list(ASPECT_DICT.keys())

inv_asp_map = {val: k for k, v in ASPECT_DICT.items() for val in v}

mapping_expr = F.create_map([F.lit(x) for x in chain(*inv_asp_map.items())])

full_asp_sdf = (
    unigram_sdf
#     unibi_sdf
#     .withColumnRenamed('unibi_word', 'tokenized_word')
    .withColumn('aspect', mapping_expr[F.col('tokenized_word')])
)
    
asp_sdf = (
    full_asp_sdf
    .dropna(subset=['aspect'])
)

In [ ]:
asp_sdf.show()

## Assign

This is gettting the "top" aspect per sentence per review, by count

In [ ]:
KEEP_COLS = [
    'normalized_url',
    'review_source_id',
    'review_content',
    'source_name',
    'tokenized_sents',
    ]

# window function not the fastest but preserves ties
window = Window.partitionBy(KEEP_COLS).orderBy(F.col('count').desc())

count_asp_sdf = (
    asp_sdf
    .groupby(KEEP_COLS+['aspect'])
    .count()
    .withColumn('rank', F.dense_rank().over(window))
)

top_asp_sdf = (
    count_asp_sdf
    .filter(F.col('rank') ==1)
    .drop('rank', 'count')
    .withColumnRenamed('aspect', 'top_asp')
)

In [ ]:
count_asp_sdf.show()

# Aspect Segmentation

The goal here is to see which terms appear disproportionately in aspects, which are NOT already part of the existing dictionary. Then, if necessary, can add those to the dictionary if it makes sense to include them.

IMPORTANT: Skip steps 14 & 15 if you feel that you have sufficient seedwords from the base list.

Chi squared value is derived from the formula using the following:
* C1 is the number of times w occurs in sentences belonging to aspect Ai
* C2 is the number of times w occurs in sentences not belonging to Ai
* C3 is the number of sentences of aspect Ai that do not contain w
* C4 is the number of sentences that neither belong to aspect Ai, nor contain word w
* C is the total number of word occurrences.

In [30]:
def chi_sq(C, C1, C2, C3, C4):
    numer = C * (C1 * C4 - C2 * C3)**2
    denom = (C1 + C3) * (C2 + C4) * (C1 + C2) * (C3 + C4)
    return numer / denom

In [ ]:
assign_sdf = full_asp_sdf.select(KEEP_COLS + ['tokenized_word', 'aspect']).join(top_asp_sdf, on=KEEP_COLS, how='left')

C = (
    full_asp_sdf
    .groupby('tokenized_word')
    .count()
)

c_pdf = C.toPandas().set_index('tokenized_word').sort_index()
c_arr = np.repeat(c_pdf.values, len(aspect_li), axis=1)

# words x aspects, times w occurs in sentences of each aspect
c1_sdf = (
    assign_sdf
    .drop('aspect')
    .withColumnRenamed('top_asp', 'aspect')
    .groupby('tokenized_word')
    .pivot('aspect', aspect_li+[None])
    .count()
    .fillna(0)
)

c3_prime_sdf = (
    assign_sdf
    .drop('aspect')
    .withColumnRenamed('top_asp', 'aspect')
    .groupby('tokenized_word')
    .pivot('aspect', aspect_li+[None])
    .agg(F.countDistinct('tokenized_sents'))
    .fillna(0)
)

c1_pdf = c1_sdf.toPandas().set_index('tokenized_word')

c1_pdf['total_count'] = c1_pdf[aspect_li+['null']].sum(axis=1)

c2_pdf = np.tile(c1_pdf[['total_count']].values, len(aspect_li)) - c1_pdf[aspect_li]

c3_prime_pdf = c3_prime_sdf.toPandas().set_index('tokenized_word')
c3_prime_pdf['total_count'] = c3_prime_pdf[aspect_li+['null']].sum(axis=1)
c3_pdf = np.tile(c3_prime_pdf[['total_count']].values, len(aspect_li)) - c3_prime_pdf[aspect_li]

col_sum = c3_prime_pdf[aspect_li].sum(axis=0)
c4_pdf = np.tile(col_sum, (len(c3_pdf),1)) - c3_pdf[aspect_li]

chi_pdf = chi_sq(
    c_arr,
    c1_pdf[aspect_li].sort_index(),
    c2_pdf[aspect_li].sort_index(),
    c3_pdf[aspect_li].sort_index(),
    c4_pdf[aspect_li].sort_index()
)

chi_melt_pdf = pd.melt(chi_pdf.reset_index(), id_vars='tokenized_word', value_vars=aspect_li)

existing_word_pdf = pd.DataFrame.from_dict(inv_asp_map, orient='index').reset_index()
existing_word_pdf.columns=['tokenized_word', 'variable']
existing_word_pdf['existing'] = True

chi_merge_pdf = chi_melt_pdf.merge(existing_word_pdf, how='left', on=['variable', 'tokenized_word'])
chi_merge_pdf['existing'] = chi_merge_pdf['existing'].fillna(False)
filtered_chi_pdf = chi_merge_pdf[chi_merge_pdf['existing']!=True]

In [ ]:
top_bootstrap_pdf = filtered_chi_pdf.sort_values(['variable', 'value'], ascending=False).groupby(['variable']).head(TOP_P)

Create dictionary to include appropriate seedwords

In [ ]:
bootstrap_dict = top_bootstrap_pdf.groupby('variable')['tokenized_word'].apply(list).to_frame()['tokenized_word'].to_dict()
add_seed_dict = {aspect: [] for aspect in aspect_li}

In [ ]:
top_bootstrap_pdf

In [ ]:
for aspect in bootstrap_dict.keys():
    input_txt =  input(f"Enter seedwords to include for KPC: {aspect} EXACTLY as displayed above, separated by a comma and space. \nThis displays words that are not included in the base dictionary \n \n").strip()

    for i in range(len(input_txt.split(', '))):
        add_seed_dict[aspect].append(input_txt.split(', ')[i])

In [ ]:
bootstrap_dict = add_seed_dict

# End Iterative Bootstrapping

if sufficient words are added to augment the seedwords, go back to [here](#Start-Iterative-Bootstrapping) and rerun the rest.

# Aggregate on Aspect

In [ ]:
# GROUP_COLS = [
#     'normalized_url',
#     'review_source_id',
#     'review_content',
# #   'source_name', # actually some of them are duplicates across sources
#     'top_asp',
#     ]

In [ ]:
clustered_asp_sdf = (
    top_asp_sdf
    .groupby(GROUP_COLS)
    .agg(
        F.collect_list(F.col('tokenized_sents')).alias('sent_li')
    )
    .withColumn('combined_sents', F.array_join(F.col('sent_li'), ' '))
    .drop('sent_li')
)

# Sentiment Intensity Analysis

Need to decide if we want to keep this method of using sentiment analysis or if we want to just use ratings

In [ ]:
sid = SIA()
@F.udf(T.DoubleType())
def get_sentiment_score(sentence):
    pscores = sid.polarity_scores(sentence)
    return pscores['compound']

In [ ]:
scored_asp_sdf = (
    clustered_asp_sdf
    .withColumn('polarity_score', get_sentiment_score(F.col('combined_sents')))
    .withColumn('overall_polarity_score', get_sentiment_score(F.col('review_content')))
)

clustered_asp_pdf = scored_asp_sdf.drop_duplicates(subset=['normalized_url', 'review_source_id']).toPandas()

overall_pdf = scored_asp_sdf[['normalized_url', 'review_source_id', 'overall_polarity_score']].drop_duplicates().toPandas()

# Pivot to Features

In [ ]:
pivot_asp = (
    clustered_asp_pdf
    .set_index(['normalized_url', 'review_source_id', 'top_asp'])[['polarity_score']]
    .unstack('top_asp')
)
pivot_asp.columns = pivot_asp.columns.get_level_values(1)
pivot_asp = pivot_asp.reset_index()
pivot_asp = pivot_asp.merge(overall_pdf, on=['normalized_url', 'review_source_id'], how='left')

for asp in aspect_li + ['overall_polarity_score']:
    pivot_asp[asp] = 3 + 2 * pivot_asp[asp] # map to [1,5] stars from [-1, 1] scale

avg_sentiment_pdf = pivot_asp.groupby('normalized_url').describe().transpose()
avg_sentiment_pdf = avg_sentiment_pdf[avg_sentiment_pdf.index != 'overall_polarity_score']

In [ ]:
pivot_asp

In [ ]:
imp_pivot_asp = pivot_asp.copy(True)
for asp in aspect_li:
    imp_pivot_asp[asp] = imp_pivot_asp[asp].fillna(pivot_asp.overall_polarity_score)

In [ ]:
# truth_pdf = brand_revs.select(aspect_li + ['review_rating', 'review_source_id']).toPandas()
truth_pdf = brand_revs.select(['normalized_url', 'review_rating', 'review_source_id']).toPandas()

feature_pdf = (
    pivot_asp
    .merge(truth_pdf, how='left', on=['normalized_url', 'review_source_id'], suffixes=('', '_true'))
    .set_index(['normalized_url', 'review_source_id'])
    .astype(float)
)

imp_feature_pdf = (
    imp_pivot_asp
    .merge(truth_pdf, how='left', on=['normalized_url', 'review_source_id'], suffixes=('', '_true'))
    .set_index(['normalized_url', 'review_source_id'])
    .astype(float)
)

In [ ]:
imp_feature_pdf

In [ ]:
# checking correlations between rating and polarity score

In [ ]:
imp_feature_pdf.groupby('normalized_url').corr('pearson')

# Regression

This regression is fitting a model of the aspect weights to the review rating; seeing which matter most?

## Overall

In [ ]:
X = imp_feature_pdf[aspect_li].values
Y = imp_feature_pdf[['review_rating']].values

In [ ]:
est = sm.OLS(Y, X)
results = est.fit()
print(results.summary())

In [ ]:
nonneg_wts = [w if w > 0 else 0 for w in results.params]
normalized_sum = __builtins__.sum(nonneg_wts)
normalized_weights = nonneg_wts / normalized_sum
normalized_bse = results.bse / normalized_sum

### Statistical significance

In [ ]:
p_vals_pdf = pd.DataFrame(results.pvalues)
p_vals_pdf.index = aspect_li
p_vals_pdf.columns = ['p_value']
p_vals_pdf['significant_at_0.05'] = p_vals_pdf['p_value'] <= 0.05
p_vals_pdf = p_vals_pdf.round(decimals=3)
p_vals_pdf

### MAPE

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
mape_overall = mean_absolute_percentage_error(Y, results.fittedvalues)
print(f'The mean absolute percentage error is {round(mape_overall,2)}%')

## Brand Level

This is then taking at the brand level, which aspects predict the overall rating most? For one brand, Texture may drive overall rating, while for others, it could be Scent

In [ ]:
mapes = []
df_holder = []
for url, data in imp_feature_pdf.groupby('normalized_url'):
    X = data[aspect_li].values
    Y = data[['review_rating']].values
    est = sm.OLS(Y, X)
    results = est.fit()
    
    pos_wts = [w if w > 0 else 0 for w in results.params]
    
    norm_sum = __builtins__.sum(pos_wts)
    norm_weights = pos_wts / norm_sum
    norm_bse = results.bse / norm_sum
    
    data = {'weights': norm_weights, 'std': norm_bse, 'p_value': results.pvalues}
    df = pd.DataFrame.from_dict(data, orient='index')
    df.columns = columns=aspect_li
    df = df.transpose().rename_axis('aspect').reset_index()
    df['normalized_url'] = url
    df['brand_name'] = df.normalized_url.map(brand_name_di)
    df_holder.append(df)
    
    mape = mean_absolute_percentage_error(Y, results.fittedvalues)
    mapes.append((brand_name_di[url], mape))

brand_wt_pdf = pd.concat(df_holder)

In [ ]:
brand_wt_pdf.head(5)

### Statistical Significance

In [ ]:
brand_stats = brand_wt_pdf[['aspect', 'p_value', 'brand_name']]
brand_stats = brand_stats.round(decimals=3)
brand_stats['significant_at_0.05'] = brand_stats['p_value'] <= 0.05
col_order = ['brand_name', 'aspect', 'p_value', 'significant_at_0.05']
brand_stats[col_order]

### MAPE

In [ ]:
mapes_pdf = pd.DataFrame(mapes)
mapes_pdf.columns = ['brand_name', 'mape']
mapes_pdf.round(decimals=2)

# Plot

Here, making a data frame with each aspect, each brand name, and its median rating as well as relative weight

In [ ]:
rating_data = avg_sentiment_pdf.stack().to_frame().reset_index()
rating_data.columns=['aspect', 'stat', 'normalized_url', 'median']
rating_data = rating_data[(rating_data['stat']=='50%') & (rating_data['aspect']!='overall_polarity_score')].drop('stat', axis=1)
rating_data['brand_name'] = rating_data['normalized_url'].map(brand_name_di)
rating_data['weights'] = rating_data['aspect'].map(dict(zip(aspect_li, normalized_weights)))
rating_data = rating_data.sort_values(['normalized_url', 'weights'], ascending=False)

In [ ]:
rating_data.head(3)

## Plot Ratings, without weights

In [ ]:
# Was having issues with Plotly producing blank outputs, so I added this (from: https://stackoverflow.com/questions/48560138/plotly-offline-iplot-gives-a-large-blank-field-as-its-output-in-jupyter-notebook)
import plotly.io as pio
pio.renderers.default='notebook'

In [ ]:
# review count
review_count_dict = imp_feature_pdf.reset_index().groupby('normalized_url')['review_source_id'].nunique().to_dict()
labels_w_counts = {
    brand_name_di[k]: f'{brand_name_di[k]} ({str(v)} reviews)'
    for k, v in review_count_dict.items()
} 

In [ ]:
fig = px.bar(
    rating_data,
    x='aspect',
    y="median",
    color='brand_name',
    text='median',
#     error_y="std",
    labels = {'brand_name': 'Brand'},
    color_discrete_sequence=CU_PLOTLY_COLOR_SEQUENCE,
    barmode='group',
)

fig.update_traces(
    textposition='outside',
    texttemplate='%{text:.2s}',
)

fig.update_layout(
    title=f"Aspect Ratings",
    xaxis_title="Aspect",
    yaxis_title="Star Ratings (Out of 5)",
    yaxis_range=[1,5]
)

fig.update_layout(
    font=Font.plot_title.value,
    plot_bgcolor="white",
    title={"x": 0.5},
    colorway=CU_PLOTLY_COLOR_SEQUENCE,
     colorscale={"sequential": CU_PLOTLY_COLORSCALE},
    coloraxis={"autocolorscale": True},
)

fig.for_each_trace(lambda t: t.update(name = labels_w_counts[t.name]))

fig.show()

## Plot rating along with weights

This is too busy, will remove standard errors

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

line_data = px.scatter(
        rating_data,
        x='aspect',
        y="median",
        color='brand_name',
        color_discrete_sequence=CU_PLOTLY_COLOR_SEQUENCE,
        category_orders={"brand_name": BRAND_NAME_LI}
    )

bar_data =  px.bar(
        brand_wt_pdf,
        x='aspect',
        y='weights',
        error_y='std',
        color='brand_name',
        barmode='group',
        color_discrete_sequence=CU_PLOTLY_COLOR_SEQUENCE,
        category_orders={"brand_name": BRAND_NAME_LI}
    )

for data in line_data.data:
    fig.add_trace(data.update(mode='markers+lines'), secondary_y=True)
    
for data in bar_data.data:
    fig.add_trace(data)

fig.update_layout(
    title=f"KPC for Select {GROUP_NAME} Brands",
    xaxis_title="Aspect",
    yaxis_title="Overall Normalized Percent Weight",
)

fig.update_xaxes(
#     categoryorder='array',
)

fig.update_yaxes(
    range=[1,5],
    title="Brand Stars Rating (out of 5)",
    secondary_y=True
)
fig.update_yaxes(
    range=[0,1],
    tickformat='.0%',
    secondary_y=False
)

fig.update_layout(
    width=1200,
    height=800,
    font=Font.plot_title.value,
    plot_bgcolor="white",
    title={"x": 0.5},
    colorway=CU_PLOTLY_COLOR_SEQUENCE,
    colorscale={"sequential": CU_PLOTLY_COLORSCALE},
    coloraxis={"autocolorscale": True},
)

fig.for_each_trace(lambda t: t.update(name = labels_w_counts[t.name]) if t.type =='scatter' else None)

fig.show()

In [ ]:
min = __builtins__.min
max = __builtins__.max

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

line_data = px.scatter(
        rating_data,
        x='aspect',
        y="median",
        color='brand_name',
        color_discrete_sequence=CU_PLOTLY_COLOR_SEQUENCE,
        category_orders={"brand_name": BRAND_NAME_LI}
    )

for data in line_data.data:
    data['line']['width'] = 5
    fig.add_trace(data.update(mode='lines+markers'), secondary_y=True)
    fig.update_traces(marker=dict(size=12,line=dict(width=2,color='DarkSlateGrey')))

fig.add_trace(
    px.bar(
        x=aspect_li,
        y=normalized_weights,
        #error_y=normalized_bse,
        color_discrete_sequence=[Color.grey_2.value],
        barmode='group',
        opacity=0.5,
    ).data[0],
)

fig.update_layout(
    title=f"Key Purchase Criteria for Select {GROUP_NAME} Brands",
    xaxis_title="Aspect",
    yaxis_title="Importance (Percent Weight)",
)

fig.update_xaxes(
#     categoryorder='total descending'
)

fig.update_yaxes(
    range=[min(rating_data['median'])-0.1, max(rating_data['median'])+0.1],
    title="Brand Stars Rating (out of 5)",
    secondary_y=True
)
fig.update_yaxes(
    range=[0, max(normalized_weights)+max(normalized_bse)],
    tickformat='.0%',
    secondary_y=False
)

fig.update_layout(
    width=1200,
    height=800,
    font=Font.plot_title.value,
    plot_bgcolor="white",
    title={"x": 0.5},
    colorway=CU_PLOTLY_COLOR_SEQUENCE,
    colorscale={"sequential": CU_PLOTLY_COLORSCALE},
    coloraxis={"autocolorscale": True},
)

fig.update_layout(
    yaxis=dict(
        titlefont=dict(
            color=Color.grey_2.value
        ),
        tickfont=dict(
            color=Color.grey_2.value
        )
    ),
)

fig.for_each_trace(lambda t: t.update(name = labels_w_counts[t.name]) if t.name else None)

fig.show()

# QA - Review after script has run

In [ ]:
brand_name_di

In [ ]:
product_name_include_di

In [ ]:
product_name_exclude_di

In [ ]:
def qa_dictionaries(brand_name_di, product_name_include_di, product_name_exclude_di):
    if len(brand_name_di.items()) > len(product_name_include_di.items()):
        print('!!! FLAG - you may be excluding brands accidently. Must add more entries to product dictionaries.')
    else:
        print('Product name include dictionary length is good')
    if len(brand_name_di.items()) > len(product_name_exclude_di.items()):
        print('!!! FLAG - you may be excluding brands accidently. Must add more entries to product dictionaries.')
    else:
        print('Product name exclude dictionary length is good')

In [ ]:
qa_dictionaries(brand_name_di, product_name_include_di, product_name_exclude_di)

In [ ]:
# # Test QA Function
# brand_name_di_test = brand_name_di
# brand_name_di_test['test_item'] = 'test'
# brand_name_di_test

In [ ]:
# # Test
# qa_dictionaries(brand_name_di_test, product_name_include_di, product_name_exclude_di)

# View DF Heads

In [ ]:
import pandas as pd
%whos DataFrame

In [ ]:
# Helpers function will create list of all dataframes in notebook in chronological order (from when they are run in the script)

from pyspark.sql import DataFrame

def list_all_dataframes():
    return [k for (k, v) in globals().items() if isinstance(v, pd.DataFrame) or isinstance(v, DataFrame)]
list_all_df = list_all_dataframes()
len(list_all_df)

In [ ]:
# Parameter defined at top of script
if print_all_dataframes == 'Y':

    # Write a function that will print both Pandas and Spark dataframes
    for df in list_all_df:
        df_name = df
        df = df.replace("'", "")
        df = locals()[df]
        print('df - ' + df_name )
        print(type(df))
        try: 
            display(df.limit(5).toPandas())
        except Exception:
            try:
                display(df.head(5))
            except:
                print('ERROR - ' + df_name)            

else:
    print('parameter indicates print_all_dataframes is not requested, change to Y at top of script if interested')